In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
import sys

Append the `grt_reader` repo so we can read modules from sibling directories.


In [30]:
sys.path.append(os.path.abspath(os.path.join("..")))

In [31]:
import numpy as np
import pandas as pd
from utils.constants import question_ids_map
from utils.process_lease import process_lease
from utils.questions import (
    construct_single_question_for_ai,
)

In [32]:
bucket_name = os.getenv("BUCKET_NAME", "")
index_name = os.getenv("INDEX_NAME", "")

In [33]:
print(f"Bucket name: {bucket_name}")
print(f"Index name: {index_name}")

Bucket name: gaaprt-reader-dev
Index name: development-gaaprt-reader-openai


Retrieve questions


In [34]:
questions_df = pd.read_excel("../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [35]:
answers_df = pd.read_excel("../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [36]:
question_id = question_ids_map["effective_date"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

In [37]:
file_name = "lease036"
local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
file_key = f"eafit/{file_name}.pdf"
namespace = f"eafit_{file_name}"

In [38]:
result = process_lease(
    local_file=local_file,
    file_name=file_name,
    file_key=file_key,
    namespace=namespace,
    question_for_ai=question_for_ai,
    question_id=question_id,
    answers_df=answers_df,
)

Upload Successful: eafit/lease036.pdf
Stored eafit/lease036.pdf in vector database with namespace eafit_lease036


In [39]:
answer = result["answer"]
real_answer_unprocessed = result["real_answer_unprocessed"]

In [40]:
formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")

In [41]:
formatted_answer == answer

True